# MLflow + Colab – Example project
This project shows how you can easily log experiments with Google Colab, directly to an MLflow remote.

It uses DAGsHub MLflow remote server, which is a free hosted MLflow remote.

Instructions:
1. [Sign up to DAGsHub](https://dagshub.com/user/sign_up)
1. [Create an access token](https://dagshub.com/user/settings/tokens)
1. [Create a new project](https://dagshub.com/repo/create)
1. [Open the notebook in Google Colab](https://dagshub.com/Dean/mlflow-colab-example/src/master/mlflow-colab.ipynb)
1. Run the cell
1. Fill in your DAGsHub username, access token and project name.
1. Run the visualization cell if you want to see your experiments inside Colab.

In [11]:
!pip install mlflow --quiet

import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ')
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('Enter your DAGsHub project name: ')

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME']
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

Enter your DAGsHub username: ibm26ashish
Enter your DAGsHub access token: ··········
Enter your DAGsHub project name: MLFlow_mnist_keras


In [13]:
"""Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
"""
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow

mlflow.tensorflow.autolog()

max_words = 1000
batch_size = 32
epochs = 25

print("Loading data...")
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

num_classes = np.max(y_train) + 1
print(num_classes, "classes")

print("Vectorizing sequence data...")
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
x_test = tokenizer.sequences_to_matrix(x_test, mode="binary")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

print("Convert class vector to binary class matrix (for use with categorical_crossentropy)")
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

print("Building model...")
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1
)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])

Loading data...
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)
Building model...


2024/02/22 07:49:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '71a2e221b88a4cdd951b679a7a95a23f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/25
253/253 [==============================] - 2s 5ms/step - loss: 1.4214 - accuracy: 0.6858 - val_loss: 1.1016 - val_accuracy: 0.7620
Epoch 2/25
253/253 [==============================] - 1s 4ms/step - loss: 0.7863 - accuracy: 0.8097 - val_loss: 0.9508 - val_accuracy: 0.7875
Epoch 3/25
253/253 [==============================] - 1s 4ms/step - loss: 0.5519 - accuracy: 0.8689 - val_loss: 0.8856 - val_accuracy: 0.8009
Epoch 4/25
253/253 [==============================] - 1s 5ms/step - loss: 0.4165 - accuracy: 0.8945 - val_loss: 0.8750 - val_accuracy: 0.8187
Epoch 5/25
253/253 [==============================] - 1s 4ms/step - loss: 0.3359 - accuracy: 0.9166 - val_loss: 0.8768 - val_accuracy: 0.8131
Epoch 6/25
253/253 [==============================] - 1s 4ms/step - loss: 0.2737 - accuracy: 0.9316 - val_loss: 0.9330 - val_accuracy: 0.8076
Epoch 7/25
253/253 [==============================] - 1s 5ms/step - loss: 0.2403 - accuracy: 0.9368 - val_loss: 0.9783 - val_accuracy: 0.8087
Epoch 

In [15]:
# See your experiments table inside Colab!
import IPython
display(IPython.display.IFrame("https://dagshub.com/"+ os.environ['MLFLOW_TRACKING_USERNAME']
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + "/experiments/#/",'100%',600))

In [ ]:
!pip list | grep mlflow

mlflow                        1.22.0
